# Einheit 3: Tiefes Q-Learning mit Atari-Spielen 👾 mit RL Baselines3 Zoo

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/thumbnail.jpg" alt="Unit 3 Thumbnail">

In diesem Notizbuch **trainieren Sie einen Deep Q-Learning-Agenten**, der Space Invaders spielt. Dazu verwenden Sie [RL Baselines3 Zoo] (https://github.com/DLR-RM/rl-baselines3-zoo), ein auf [Stable-Baselines3] (https://stable-baselines3.readthedocs.io/en/master/) basierendes Trainingsframework, das Skripte für das Training, die Auswertung von Agenten, die Abstimmung von Hyperparametern, die Darstellung von Ergebnissen und die Aufnahme von Videos bereitstellt.

Wir verwenden die [RL-Baselines-3 Zoo Integration, eine Vanilla-Version von Deep Q-Learning](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html) ohne Erweiterungen wie Double-DQN, Dueling-DQN und Prioritized Experience Replay.

⬇️ Hier ein Beispiel dafür, was **Sie erreichen werden** ⬇️

In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/ThomasSimonini/ppo-SpaceInvadersNoFrameskip-v4/resolve/main/replay.mp4" type="video/mp4"></video>

### 🎮 Umgebungen:

- [SpacesInvadersNoFrameskip-v4](https://gymnasium.farama.org/environments/atari/space_invaders/)

Sie können den Unterschied zwischen den Space Invaders Versionen hier sehen 👉 https://gymnasium.farama.org/environments/atari/space_invaders/#variants

### 📚 RL-Library:

- [RL-Baselines3-Zoo](https://github.com/DLR-RM/rl-baselines3-zoo)

## Ziele dieses Notizbuchs 🏆
Am Ende des Heftes wirst du:
- In der Lage sein, tiefer zu verstehen **wie RL Baselines3 Zoo funktioniert**.
- In der Lage sein, **Ihren trainierten Agenten und den Code in den Hub** mit einer schönen Videowiedergabe und einem Bewertungsergebnis zu pushen 🔥.




## Dieses Notebook stammt aus dem Deep Reinforcement Learning Kurs.
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

In diesem kostenlosen Kurs lernen Sie:

- 📖 Deep Reinforcement Learning in **Theorie und Praxis** studieren.
- 🧑‍💻 Lernen Sie, **berühmte Deep RL-Bibliotheken** wie Stable Baselines3, RL Baselines3 Zoo, CleanRL und Sample Factory 2.0 zu verwenden.
- 🤖 Trainieren Sie **Agenten in einzigartigen Umgebungen**.

Und mehr, siehe 📚 den Lehrplan 👉 https://simoninithomas.github.io/deep-rl-course

Vergessen Sie nicht, sich **<a href="http://eepurl.com/ic5ZUD">für den Kurs anzumelden</a>** (wir sammeln Ihre E-Mail, um Ihnen **die Links zu senden, wenn die einzelnen Einheiten veröffentlicht werden, und Sie über die Herausforderungen und Aktualisierungen zu informieren).**


Der beste Weg, um in Kontakt zu bleiben, ist, unserem Discord-Server beizutreten, um sich mit der Community und mit uns auszutauschen 👉🏻 https://discord.gg/ydHrjt3WP5

# RL-Baselines3 Zoo und seine Abhängigkeiten installieren 📚.

Wenn Sie "ERROR: pip's dependency resolver does not currently take into account all the packages that are installed" sehen, **das ist normal und kein kritischer Fehler**, gibt es einen Versionskonflikt. Aber die Pakete, die wir brauchen, sind installiert.

In [ ]:
# For now we install this update of RL-Baselines3 Zoo
!pip install git+https://github.com/DLR-RM/rl-baselines3-zoo@update/hf

WENN UND NUR WENN DIE OBIGE VERSION NICHT MEHR EXISTIERT. ENTKOMMENTIEREN UND INSTALLIEREN SIE DIE UNTEN STEHENDE VERSION

In [ ]:
#!pip install rl_zoo3==2.0.0a9

In [ ]:
!apt-get install swig cmake ffmpeg

Um Atari-Spiele in Gymnasium verwenden zu können, müssen wir das Atari-Paket installieren. Und accept-rom-license, um die Rom-Dateien (Spiele-Dateien) herunterzuladen.

In [ ]:
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]

## Erstellen einer virtuellen Anzeige 🔽.

Während der Arbeit mit dem Notebook müssen wir ein Wiederholungsvideo erstellen. Dazu benötigen wir mit colab **einen virtuellen Bildschirm, um die Umgebung zu rendern** (und somit die Bilder aufzunehmen).

Daher wird die folgende Zelle die Librairies installieren und einen virtuellen Bildschirm erstellen und starten 🖥

In [ ]:
%%capture
!apt install python-opengl
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# Von Q-Learning zu Deep Q-Learning

Wir haben gelernt, dass **Q-Learning ein Algorithmus ist, mit dem wir unsere Q-Funktion** trainieren, eine **Aktionswertfunktion**, die den Wert eines bestimmten Zustands und einer bestimmten Aktion in diesem Zustand bestimmt.

<figure>
  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-function.jpg" alt="Q-function"/>
</figure>

Das **Q kommt von "der Qualität" dieser Handlung in diesem Zustand.**

Intern wird unsere Q-Funktion durch eine **Q-Tabelle kodiert, eine Tabelle, in der jede Zelle einem Wert eines Zustands-Aktionspaares entspricht.** Man kann sich diese Q-Tabelle als **den Speicher oder Spickzettel unserer Q-Funktion vorstellen.**

Das Problem ist, dass Q-Learning eine *tabellarische Methode* ist. Dies wird zu einem Problem, wenn die Zustände und Aktionsräume **nicht klein genug sind, um effizient durch Arrays und Tabellen dargestellt zu werden**. Mit anderen Worten: es ist **nicht skalierbar**.
Q-Learning funktionierte gut mit kleinen Zustandsräumen wie:

- FrozenLake, wir hatten 16 Zustände.
- Taxi-v3, wir hatten 500 Zustände.

Aber stellen Sie sich vor, was wir heute tun werden: Wir werden einen Agenten trainieren, damit er lernt, Space Invaders zu spielen, ein komplexeres Spiel, bei dem die Frames als Input dienen.

Wie **[Nikita Melkozerov] (https://twitter.com/meln1k) erwähnte, haben Atari-Umgebungen** einen Beobachtungsraum mit der Form (210, 160, 3)*, der Werte von 0 bis 255 enthält, so dass wir \\(256^{210 \times 160 \times 3} = 256^{100800}\\) mögliche Beobachtungen haben (zum Vergleich: wir haben ungefähr \\\(10^{80}\) Atome im beobachtbaren Universum).

* Ein Einzelbild in Atari besteht aus einem Bild von 210x160 Pixeln. Da die Bilder in Farbe (RGB) sind, gibt es 3 Kanäle. Aus diesem Grund ist die Form (210, 160, 3). Für jedes Pixel kann der Wert von 0 bis 255 gehen.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/atari.jpg" alt="Atari State Space"/>

Der Zustandsraum ist also gigantisch; aus diesem Grund wäre das Erstellen und Aktualisieren einer Q-Tabelle für diese Umgebung nicht effizient. In diesem Fall ist die beste Idee, die Q-Werte mit Hilfe einer parametrisierten Q-Funktion \\(Q_{\theta}(s,a)\\) zu approximieren.

Dieses neuronale Netz approximiert für einen Zustand die verschiedenen Q-Werte für jede mögliche Aktion in diesem Zustand. Und das ist genau das, was Deep Q-Learning tut.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/deep.jpg" alt="Deep Q-Learning"/>


Nachdem wir nun Deep Q-Learning verstanden haben, wollen wir nun tiefer in das Deep Q-Network eintauchen.

# Das Deep Q-Network (DQN)
Dies ist die Architektur unseres Deep Q-Learning-Netzwerks:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/deep-q-network.jpg" alt="Deep Q Network"/>

Als Eingabe nehmen wir einen **Stapel von 4 Einzelbildern**, der als Zustand durch das Netz läuft, und geben einen **Vektor von Q-Werten für jede mögliche Aktion in diesem Zustand** aus. Dann müssen wir, wie beim Q-Learning, nur noch unsere Epsilon-Greedy-Strategie verwenden, um die zu ergreifende Aktion auszuwählen.

Bei der Initialisierung des neuronalen Netzes ist **die Schätzung der Q-Werte miserabel**. Aber während des Trainings wird unser Deep Q-Network-Agent eine Situation mit der passenden Aktion assoziieren und **erlernen, das Spiel gut zu spielen**.

## Vorverarbeitung der Eingabe und zeitliche Begrenzung

Wir müssen die Eingabe **vorverarbeiten**. Dies ist ein wesentlicher Schritt, da wir die Komplexität unseres Zustands **reduzieren wollen, um die für das Training benötigte Rechenzeit zu verringern**.

Um dies zu erreichen, **reduzieren wir den Zustandsraum auf 84x84 und graustufen ihn**. Das können wir tun, da die Farben in Atari-Umgebungen keine wichtigen Informationen hinzufügen.
Dies ist eine große Verbesserung, da wir **unsere drei Farbkanäle (RGB) auf 1 reduzieren**.

In einigen Spielen können wir auch **einen Teil des Bildschirms abschneiden**, wenn er keine wichtigen Informationen enthält.
Dann stapeln wir vier Bilder zusammen.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/preprocessing.jpg" alt="Preprocessing"/>

**Warum stapeln wir vier Frames zusammen?**
Wir stapeln Bilder zusammen, weil es uns hilft, **das Problem der zeitlichen Begrenzung** zu lösen. Nehmen wir ein Beispiel mit dem Spiel Pong. Wenn Sie dieses Bild sehen:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/temporal-limitation.jpg" alt="Zeitliche Begrenzung"/>

Können Sie mir sagen, wohin der Ball fliegt?
Nein, denn ein Bild reicht nicht aus, um ein Gefühl für die Bewegung zu bekommen! Aber was ist, wenn ich drei weitere Bilder hinzufüge? **Hier kannst du sehen, dass der Ball nach rechts geht**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/temporal-limitation-2.jpg" alt="Zeitliche Begrenzung"/>
Um die zeitlichen Informationen zu erfassen, werden daher vier Bilder übereinander gelegt.

Dann werden die gestapelten Bilder von drei Faltungsschichten verarbeitet. Diese Schichten **ermöglichen es uns, räumliche Beziehungen in Bildern zu erfassen und zu nutzen**. Da die Bilder übereinander gestapelt sind, können wir **aber auch einige zeitliche Eigenschaften über diese Bilder hinweg ausnutzen**.

Wenn Sie nicht wissen, was Faltungsschichten sind, machen Sie sich keine Sorgen. Sie können sich [Lektion 4 dieses kostenlosen Deep Learning-Kurses von Udacity] ansehen (https://www.udacity.com/course/deep-learning-pytorch--ud188)

Schließlich haben wir ein paar vollständig verbundene Schichten, die einen Q-Wert für jede mögliche Aktion in diesem Zustand ausgeben.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/deep-q-network.jpg" alt="Deep Q Network"/>

Wir sehen also, dass Deep Q-Learning ein neuronales Netz verwendet, um für einen Zustand die verschiedenen Q-Werte für jede mögliche Aktion in diesem Zustand zu approximieren. Schauen wir uns nun den Deep Q-Learning-Algorithmus an.


# Der tiefe Q-Learning-Algorithmus

Wir haben gelernt, dass Deep Q-Learning **ein tiefes neuronales Netz verwendet, um die verschiedenen Q-Werte für jede mögliche Aktion in einem Zustand** zu approximieren (Wert-Funktions-Schätzung).

Der Unterschied besteht darin, dass während der Trainingsphase der Q-Wert eines Zustands-Aktionspaares nicht direkt aktualisiert wird, wie wir es beim Q-Learning getan haben:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/q-ex-5.jpg" alt="Q Loss"/>

Beim Deep Q-Learning erstellen wir eine **Verlustfunktion, die unsere Q-Wert-Vorhersage mit dem Q-Ziel vergleicht und den Gradientenabstieg verwendet, um die Gewichte unseres Deep Q-Netzes zu aktualisieren, um unsere Q-Werte besser anzunähern**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/Q-target.jpg" alt="Q-Ziel"/>

Der Deep Q-Learning-Trainingsalgorithmus hat *zwei Phasen*:

- **Sampling**: Wir führen Aktionen durch und **speichern die beobachteten Erfahrungstupel in einem Wiedergabespeicher**.
- **Training**: Wir wählen einen **kleinen Stapel von Tupeln nach dem Zufallsprinzip aus und lernen aus diesem Stapel mit Hilfe eines Aktualisierungsschritts nach dem Gradientenabstieg**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/sampling-training.jpg" alt="Sampling Training"/>

Dies ist nicht der einzige Unterschied zum Q-Learning. Das tiefe Q-Learning-Training **kann unter Instabilität leiden**, vor allem wegen der Kombination einer nichtlinearen Q-Wert-Funktion (Neuronales Netz) und Bootstrapping (wenn wir Ziele mit vorhandenen Schätzungen und nicht mit einer tatsächlichen vollständigen Rückkehr aktualisieren).

Um das Training zu stabilisieren, implementieren wir drei verschiedene Lösungen:
1. *Experience Replay* für eine **effizientere Nutzung von Erfahrungen**.
2. *Fixed Q-Target* **zur Stabilisierung des Trainings**.
3. *Double Deep Q-Learning*, um **das Problem der Überschätzung von Q-Werten** zu lösen.

Gehen wir sie durch!



## Erfahrungswiederholung für eine effizientere Nutzung von Erfahrungen

Warum erstellen wir einen Wiedergabespeicher?

Die Erfahrungswiederholung beim Deep Q-Learning hat zwei Funktionen:

1. **Eine effizientere Nutzung der Erfahrungen während des Trainings**.
Normalerweise interagiert der Agent beim Online-Verstärkungslernen mit der Umgebung, sammelt Erfahrungen (Zustand, Aktion, Belohnung und nächster Zustand), lernt aus ihnen (aktualisiert das neuronale Netz) und verwirft sie. Dies ist nicht effizient.

Die Erfahrungswiederholung hilft, indem sie **die Erfahrungen aus dem Training effizienter nutzt**. Wir verwenden einen Wiederholungspuffer, der Erfahrungsproben speichert, **die wir während des Trainings wiederverwenden können**.
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/experience-replay.jpg" alt="Erfahrungswiederholung"/>

⇒ Dies ermöglicht es dem Agenten, **mehrmals aus denselben Erfahrungen zu lernen**.

2. **Vermeiden Sie das Vergessen früherer Erfahrungen (auch bekannt als katastrophale Interferenz oder katastrophales Vergessen) und reduzieren Sie die Korrelation zwischen Erfahrungen**.
- **[katastrophales Vergessen](https://en.wikipedia.org/wiki/Catastrophic_interference)**: Das Problem, das sich ergibt, wenn wir unserem neuronalen Netz aufeinanderfolgende Muster von Erfahrungen geben, ist, dass es dazu neigt, **die vorherigen Erfahrungen zu vergessen, wenn es neue Erfahrungen macht**. Wenn der Agent beispielsweise in der ersten Ebene ist und dann in der zweiten, die anders ist, kann er vergessen, wie er sich in der ersten Ebene verhalten und spielen soll.

Die Lösung besteht darin, einen Wiederholungspuffer zu erstellen, der Erfahrungstupel speichert, während er mit der Umgebung interagiert, und dann eine kleine Menge von Tupeln abzufragen. Dadurch wird verhindert, dass **das Netzwerk nur über das lernt, was es unmittelbar zuvor getan hat.**

Die Wiederholung von Erfahrungen hat noch weitere Vorteile. Indem wir die Erfahrungen nach dem Zufallsprinzip abfragen, beseitigen wir Korrelationen in den Beobachtungssequenzen und verhindern, dass **Aktionswerte oszillieren oder in katastrophaler Weise divergieren.**

Im Pseudocode von Deep Q-Learning **initialisieren wir einen Wiedergabespeicher D mit der Kapazität N** (N ist ein Hyperparameter, den Sie definieren können). Wir speichern dann Erfahrungen im Speicher und nehmen eine Reihe von Erfahrungen auf, um das Deep Q-Netz während der Trainingsphase zu füttern.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/experience-replay-pseudocode.jpg" alt="Experience Replay Pseudocode"/>

## Festes Q-Ziel zur Stabilisierung des Trainings 

Wenn wir den TD-Fehler (auch bekannt als Verlust) berechnen wollen, berechnen wir die **Differenz zwischen dem TD-Ziel (Q-Ziel) und dem aktuellen Q-Wert (Schätzung von Q)**.

Aber wir haben **keine Ahnung vom tatsächlichen TD-Ziel**. Wir müssen es schätzen. Anhand der Bellman-Gleichung haben wir gesehen, dass das TD-Ziel nur die Belohnung für die Durchführung dieser Aktion in diesem Zustand plus den abgezinsten höchsten Q-Wert für den nächsten Zustand ist.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/Q-target.jpg" alt="Q-Ziel"/>

Das Problem ist jedoch, dass wir dieselben Parameter (Gewichte) für die Schätzung des TD-Ziels **und** des Q-Werts verwenden. Folglich gibt es eine signifikante Korrelation zwischen dem TD-Ziel und den Parametern, die wir ändern.

Daher verschieben sich bei jedem Trainingsschritt **sowohl unsere Q-Werte als auch die Zielwerte.** Wir kommen unserem Ziel näher, aber das Ziel bewegt sich auch. Es ist, als würden wir einem beweglichen Ziel hinterherjagen! Dies kann zu erheblichen Schwankungen im Training führen.

Es ist so, als wären Sie ein Cowboy (die Q-Schätzung) und wollten eine Kuh fangen (das Q-Ziel). Ihr Ziel ist es, näher heranzukommen (den Fehler zu verringern).

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/qtarget-1.jpg" alt="Q-Ziel"/>

In jedem Zeitschritt versuchst du, dich der Kuh zu nähern, die sich ebenfalls in jedem Zeitschritt bewegt (weil du die gleichen Parameter verwendest).

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/qtarget-2.jpg" alt="Q-Ziel"/>
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/qtarget-3.jpg" alt="Q-Ziel"/>
Dies führt zu einem bizarren Pfad der Verfolgung (eine signifikante Oszillation im Training).
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/qtarget-4.jpg" alt="Q-Ziel"/>

Stattdessen sehen wir im Pseudocode, dass wir:
- Verwendung eines **separaten Netzwerks mit festen Parametern** zur Schätzung des TD-Targets
- **Kopieren Sie die Parameter aus unserem tiefen Q-Netz alle C-Schritte**, um das Zielnetz zu aktualisieren.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/fixed-q-target-pseudocode.jpg" alt="Fixed Q-target Pseudocode"/>



## Doppeltes DQN 
Double DQNs, oder Double Deep Q-Learning neuronale Netze, wurden [von Hado van Hasselt](https://papers.nips.cc/paper/3964-double-q-learning) eingeführt. Diese Methode **behandelt das Problem der Überschätzung der Q-Werte**.

Um dieses Problem zu verstehen, erinnern wir uns daran, wie wir das TD Target berechnen:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/TD-1.jpg" alt="TD target"/>

Bei der Berechnung des TD-Ziels stehen wir vor einem einfachen Problem: Wie können wir sicher sein, dass **die beste Aktion für den nächsten Zustand die Aktion mit dem höchsten Q-Wert ist?**

Wir wissen, dass die Genauigkeit der Q-Werte davon abhängt, welche Aktion wir ausprobiert haben **und** welche benachbarten Zustände wir erkundet haben.

Folglich haben wir zu Beginn des Trainings nicht genügend Informationen über die beste Aktion, die wir durchführen sollten. Daher kann die Annahme des maximalen Q-Werts (der verrauscht ist) als beste Aktion zu falsch positiven Ergebnissen führen. Wenn nicht-optimalen Aktionen regelmäßig **ein höherer Q-Wert als der optimalen besten Aktion zugewiesen wird, wird das Lernen kompliziert**.

Die Lösung lautet: Bei der Berechnung des Q-Ziels verwenden wir zwei Netze, um die Aktionsauswahl von der Erzeugung des Q-Zielwerts zu entkoppeln. Wir:
- Wir verwenden unser **DQN-Netz**, um die beste Aktion für den nächsten Zustand zu wählen (die Aktion mit dem höchsten Q-Wert).
- Wir verwenden unser **Zielnetz**, um den Ziel-Q-Wert für die Durchführung dieser Aktion im nächsten Zustand zu berechnen.

Daher hilft uns Double DQN dabei, die Überschätzung der Q-Werte zu reduzieren und folglich schneller und stabiler zu lernen.

Seit diesen drei Verbesserungen beim Deep Q-Learning sind viele weitere hinzugekommen, wie z. B. Prioritized Experience Replay und Dueling Deep Q-Learning. Sie liegen außerhalb des Rahmens dieses Kurses, aber wenn Sie daran interessiert sind, lesen Sie die Links in der Leseliste.



## Voraussetzungen 🏗️
Bevor Sie sich mit dem Notebook beschäftigen, müssen Sie:

🔲 📚 **[Deep Q-Learning durch Lesen von Einheit 3](https://huggingface.co/deep-rl-course/unit3/introduction)** 🤗

Wir versuchen ständig, unsere Anleitungen zu verbessern. **Wenn Sie also Probleme in diesem Notizbuch** finden, öffnen Sie bitte [ein Problem im Github Repo](https://github.com/huggingface/deep-rl-class/issues).

# Trainieren wir einen Deep-Q-Learning-Agenten, der Atari' Space Invaders 👾 spielt, und laden wir ihn in den Hub hoch.

Wir empfehlen den Schülerinnen und Schülern dringend, **Google Colab für die praktischen Übungen zu verwenden, anstatt sie auf ihren Computern auszuführen**.

Durch die Verwendung von Google Colab können **Sie sich auf das Lernen und Experimentieren konzentrieren, ohne sich um die technischen Aspekte der Einrichtung Ihrer Umgebungen zu kümmern**.

Um diese praktische Übung für den Zertifizierungsprozess zu validieren, müssen Sie Ihr trainiertes Modell an den Hub senden und **ein Ergebnis von >= 200** erhalten.

Um Ihr Ergebnis zu ermitteln, gehen Sie zum Leaderboard und suchen Sie Ihr Modell, **das Ergebnis = mean_reward - std of reward**

Weitere Informationen über den Zertifizierungsprozess finden Sie in diesem Abschnitt 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

# Trainiere unseren Deep Q-Learning Agent, um Space Invaders zu spielen 👾

Um einen Agenten mit RL-Baselines3-Zoo zu trainieren, müssen wir nur zwei Dinge tun:

1. Erstellen Sie eine Hyperparameter-Konfigurationsdatei, die unsere Trainings-Hyperparameter mit dem Namen "dqn.yml" enthalten wird.

Dies ist ein Beispiel für eine Vorlage:

```
SpaceInvadersNoFrameskip-v4:
  env_wrapper:
    - stable_baselines3.common.atari_wrappers.AtariWrapper
  frame_stack: 4
  policy: 'CnnPolicy'
  n_timesteps: !!float 1e7
  buffer_size: 100000
  learning_rate: !!float 1e-4
  batch_size: 32
  learning_starts: 100000
  target_update_interval: 1000
  train_freq: 4
  gradient_steps: 1
  exploration_fraction: 0.1
  exploration_final_eps: 0.01
  # If True, you need to deactivate handle_timeout_termination
  # in the replay_buffer_kwargs
  optimize_memory_usage: False
```

Hier sehen wir das:
- Wir verwenden den "Atari Wrapper", der die Eingabe vorverarbeitet (Frame-Reduktion, Graustufen, 4 Frames stapeln)
- Wir verwenden `CnnPolicy`, da wir Faltungsschichten zur Verarbeitung der Frames verwenden
- Wir trainieren es für 10 Millionen `n_Zeitschritte`
- Die Größe des Speichers (Experience Replay) beträgt 100000, d.h. die Anzahl der Erfahrungsschritte, die Sie gespeichert haben, um Ihren Agenten erneut zu trainieren.

💡 Mein Rat ist, **die Trainingszeitschritte auf 1M zu reduzieren,** was auf einem P100 etwa 90 Minuten dauern wird. !nvidia-smi" wird Ihnen sagen, welche GPU Sie verwenden. Bei 10 Millionen Schritten wird dies etwa 9 Stunden dauern, was wahrscheinlich zu einer Zeitüberschreitung von Colab führen könnte. Ich empfehle, dies auf Ihrem lokalen Computer (oder irgendwo anders) auszuführen. Klicken Sie einfach auf: Datei>Download".

Was die Optimierung der Hyperparameter angeht, so rate ich, sich auf diese 3 Hyperparameter zu konzentrieren:
- `Lernrate`
- Puffergröße (Erfahrungsspeichergröße)`
- `Stapelgröße`

Als gute Praxis sollten Sie **die Dokumentation lesen, um zu verstehen, was jeder Hyperparameter bewirkt**: https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html#parameters



2. Wir starten das Training und speichern die Modelle im Ordner "Logs" 📁.

- Wir definieren den Algorithmus nach `--algo`, speichern das Modell nach `-f` und die Konfiguration der Hyperparameter nach `-c`.

In [ ]:
!python -m rl_zoo3.train --algo ________ --env SpaceInvadersNoFrameskip-v4  -f _________  -c _________

#### Lösung

In [ ]:
!python -m rl_zoo3.train --algo dqn  --env SpaceInvadersNoFrameskip-v4 -f logs/ -c dqn.yml

## Lass uns unseren Agenten auswerten 👀
- RL-Baselines3-Zoo bietet `enjoy.py`, ein Python-Skript zur Auswertung unseres Agenten. In den meisten RL-Bibliotheken nennen wir das Auswertungsskript `enjoy.py`.
- Lassen wir es für 5000 Zeitschritte auswerten 🔥.

In [ ]:
!python -m rl_zoo3.enjoy  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --no-render  --n-timesteps _________  --folder logs/ 

#### Lösung

In [ ]:
!python -m rl_zoo3.enjoy  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --no-render  --n-timesteps 5000  --folder logs/

## Veröffentliche unser trainiertes Modell auf dem Hub 🚀
Da wir nun gesehen haben, dass wir nach dem Training gute Ergebnisse erzielt haben, können wir unser trainiertes Modell mit einer Zeile Code auf dem Hub 🤗 veröffentlichen.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit3/space-invaders-model.gif" alt="Space Invaders model">

Mit `rl_zoo3.push_to_hub` **werten Sie aus, zeichnen ein Replay auf, generieren eine Modellkarte Ihres Agenten und schieben sie zum Hub**.

Auf diese Weise:
- Sie können **unsere Arbeit vorführen** 🔥.
- Sie können **Ihren Agenten beim Spielen visualisieren** 👀
- Du kannst **einen Agenten mit der Community teilen, den andere benutzen können** 💾
- Sie können **auf eine Bestenliste 🏆 zugreifen, um zu sehen, wie gut Ihr Agent im Vergleich zu Ihren Klassenkameraden abschneidet** 👉 https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard

Um Ihr Modell mit der Gemeinschaft teilen zu können, sind drei weitere Schritte erforderlich:

1️⃣ (Falls noch nicht geschehen) Erstellen Sie ein Konto für HF ➡ https://huggingface.co/join

2️⃣ Melde dich an und speichere dann dein Authentifizierungs-Token von der Hugging Face Website.
- Erstellen Sie ein neues Token (https://huggingface.co/settings/tokens) **mit Schreibrolle**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="HF-Token erstellen">

- Kopieren Sie das Token
- Führen Sie die Zelle unten aus und fügen Sie das Token ein

In [ ]:
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
notebook_login()
!git config --global credential.helper store

Wenn Sie kein Google Colab oder ein Jupyter Notebook verwenden möchten, müssen Sie stattdessen diesen Befehl verwenden: `huggingface-cli login`

3️⃣ Wir sind jetzt bereit, unseren geschulten Agenten zum 🤗 Hub 🔥 zu bringen.

Führen wir die Datei push_to_hub.py aus, um unseren trainierten Agenten in den Hub hochzuladen.

`-Repo-Name`: Der Name des Repo

`-orga`: Ihr Umarmungsgesicht-Benutzername

`-f`: Wo sich der Ordner des trainierten Modells befindet (in unserem Fall `logs`)

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit3/select-id.png" alt="Id auswählen">

In [ ]:
!python -m rl_zoo3.push_to_hub  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --repo-name _____________________ -orga _____________________ -f logs/

#### Lösung

In [ ]:
!python -m rl_zoo3.push_to_hub  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --repo-name dqn-SpaceInvadersNoFrameskip-v4  -orga ThomasSimonini  -f logs/

###.

Herzlichen Glückwunsch 🥳 Sie haben gerade Ihren ersten Deep Q-Learning-Agenten mit RL-Baselines-3 Zoo trainiert und hochgeladen. Das obige Skript sollte einen Link zu einem Modell-Repository wie https://huggingface.co/ThomasSimonini/dqn-SpaceInvadersNoFrameskip-v4 angezeigt haben. Wenn Sie zu diesem Link gehen, können Sie:

- Eine **Video-Vorschau Ihres Agenten** auf der rechten Seite sehen.
- Klicken Sie auf "Dateien und Versionen", um alle Dateien im Repository zu sehen.
- Klicken Sie auf "Use in stable-baselines3", um ein Codeschnipsel zu erhalten, das zeigt, wie man das Modell lädt.
- Eine Modellkarte (Datei `README.md`), die eine Beschreibung des Modells und der verwendeten Hyperparameter enthält.

Unter der Haube verwendet der Hub git-basierte Repositories (keine Sorge, wenn Sie nicht wissen, was git ist), was bedeutet, dass Sie das Modell mit neuen Versionen aktualisieren können, wenn Sie experimentieren und Ihren Agenten verbessern.

**Vergleiche die Ergebnisse deiner Agenten mit denen deiner Klassenkameraden** mit Hilfe des [leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) 🏆

## Laden Sie ein leistungsstarkes trainiertes Modell 🔥.
- Das Stable-Baselines3-Team hat **mehr als 150 trainierte Deep Reinforcement Learning-Agenten auf den Hub** hochgeladen.

Sie können sie hier finden: 👉 https://huggingface.co/sb3

Einige Beispiele:
- Asteroiden: https://huggingface.co/sb3/dqn-AsteroidsNoFrameskip-v4
- Beam Rider: https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4
- Breakout: https://huggingface.co/sb3/dqn-BreakoutNoFrameskip-v4
- Road Runner: https://huggingface.co/sb3/dqn-RoadRunnerNoFrameskip-v4

Laden wir einen Agenten, der Beam Rider spielt: https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4

In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4/resolve/main/replay.mp4" type="video/mp4"></video>

1. Wir laden das Modell mit `rl_zoo3.load_from_hub` herunter und legen es in einem neuen Ordner ab, den wir `rl_trained` nennen können

In [ ]:
# Download model and save it into the logs/ folder
!python -m rl_zoo3.load_from_hub --algo dqn --env BeamRiderNoFrameskip-v4 -orga sb3 -f rl_trained/

2. Lassen Sie uns für 5000 Zeitschritte auswerten, ob

In [ ]:
!python -m rl_zoo3.enjoy --algo dqn --env BeamRiderNoFrameskip-v4 -n 5000  -f rl_trained/ --no-render

Warum nicht versuchen, Ihre eigenen **Deep Q-Learning Agent spielen BeamRiderNoFrameskip-v4 zu trainieren? 🏆.**

Wenn Sie versuchen wollen, überprüfen Sie https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4#hyperparameters **in der Modellkarte, haben Sie die Hyperparameter des trainierten Agenten.**

Aber die Suche nach Hyperparametern kann eine schwierige Aufgabe sein. Glücklicherweise werden wir in der nächsten Einheit sehen, wie wir **Optuna für die Optimierung der Hyperparameter 🔥 verwenden können.**


## Einige zusätzliche Herausforderungen 🏆
Die beste Art zu lernen **ist, Dinge selbst auszuprobieren**!

In der [Rangliste] (https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) findest du deine Agenten. Kannst du es an die Spitze schaffen?

Hier ist eine Liste von Umgebungen, mit denen du deinen Agenten trainieren kannst:
- BeamRiderNoFrameskip-v4
- BreakoutKeinFrameskip-v4
- EnduroKeinFrameskip-v4
- PongNoFrameskip-v4

Außerdem **wenn Sie lernen wollen, Deep Q-Learning selbst zu implementieren**, sollten Sie sich unbedingt die CleanRL-Implementierung ansehen: https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/dqn_atari.py

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/atari-envs.gif" alt="Umgebungen"/>

________________________________________________________________________
Herzlichen Glückwunsch zum Abschluss dieses Kapitels!

Wenn du dich immer noch verwirrt fühlst mit all diesen Elementen...das ist völlig normal! **So ging es mir und allen anderen, die RL studiert haben.

Nimm dir Zeit, um den Stoff wirklich zu **verstehen, bevor du weitermachst und die zusätzlichen Herausforderungen ausprobierst**. Es ist wichtig, diese Elemente zu beherrschen und eine solide Grundlage zu haben.

In der nächsten Einheit **werden wir etwas über [Optuna](https://optuna.org/)** lernen. Eine der wichtigsten Aufgaben beim Deep Reinforcement Learning ist es, einen guten Satz von Trainingshyperparametern zu finden. Und Optuna ist eine Bibliothek, die Ihnen hilft, die Suche zu automatisieren.






### Dies ist ein Kurs, der mit Ihnen aufgebaut wurde 👷🏿‍♀️

Schließlich wollen wir den Kurs mit Hilfe Ihres Feedbacks verbessern und aktualisieren. Wenn Sie welche haben, füllen Sie bitte dieses Formular aus 👉 https://forms.gle/3HgA7bEHwAmmLfwh9

Wir versuchen ständig, unsere Tutorials zu verbessern. **Wenn Sie also Probleme in diesem Notizbuch finden**, öffnen Sie bitte [einen Fehler im Github Repo] (https://github.com/huggingface/deep-rl-class/issues).

Wir sehen uns bei Bonuseinheit 2! 🔥

### Keep Learning, Stay Awesome 🤗